# NETCONF/YANG
This notebook goes through a set of examples with a live platform. The platform should be running IOS-XE 16.3.2. The goal is to show how NETCONF/YANG can be leveraged to perform a range of tasks. We will cover topics like:
* Basic connectivity
* Why we really want to use some form of client library
* Getting started with a Python client like ncclient
* Capabilities discovery
* Model discovery 
* Feature discovery

First, let's talk a little about the environment being used here. This is all run inside jupyter notebooks, which some may already be familiar with as iPython notebooks. A jupyter notebook lets us mix text and code and provides a simple way to experiment with Python code. 

<span style="color:red">**IMPORTANT for Execution. PLEASE READ**</span>. At its simplest, this is a playbook and execution is top-down. Code cells will look something like "In [#]". That's where you want to single-click. To execute the code, simply hit Shift+Return. The code should execute, and move onto the next cell. It's easy to "execute" an entire notebook, task-by-task, buy continuing to execute Shift+Return. Also, if you ever see "In [*]" in your code cells, it just means things are executing, and you are probably waiting on data to come back. So depending on what you're doing, some patience might be required.

But as well as top-down execution, you can go back to code cells you just executed, tweak the code and run again! This makes it an ideal environment for experimentation and self-education.
    

### Connecting to a Device
First, we need to make sure the device itself has NETCONF/YANG enabled. It's pretty simple:

``` html
o22-3850-1#conf t
Enter configuration commands, one per line.  End with CNTL/Z.
o22-3850-1(config)#netconf-yang
o22-3850-1(config)#```

Yes, it's that simple! To reference the internal user-guide for pre-requisites though, please see here.

Next, let's define some variables that let us attach to the device. First, let's connect through an SSH tunnel to a device running in the lab. You should replace the variables below with the device of your choice:



In [9]:
# This is the router you will use for this example notebook, and maybe others.
HOST = '172.26.170.252' (replace with your own IP or your own network device)
PORT = 830
USER = 'admin' (replace with your own user configured on your network device)
PASS = 'password' (replace with .. you get the drift)

As you can see, NETCONF runs on port 830, via SSH, and needs credentials. Before you ask, these credentials can and should be stored on a AAA server, and not on the router/switch directly. Just depends on how the router is setup in advance. NOTE: In IOS-XE 16.3, keys cannot yet be used. 

Now let's establish a NETCONF session to that box using ncclient.

In [10]:
from ncclient import manager
from lxml import etree

def pretty_print(nc_retval):
    '''The ncclient library works with XML. It takes XML in and gives XML back.
    Yeah, I know it's not JSON, so we need to make it a little easier to read,
    so let's define a pretty printer we can use with ncclient responses
    '''
    print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))
    
def my_unknown_host_cb(host, fingerprint):
    return True

m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    timeout=600,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

But what is this, and what did it do? It imports the manager class from the ncclient module. It imports whatever functions are in the modules and you can import the namespace into your current namespace (like the notebook here, or in real life, an actual program). Same can be said for the etree library from lxml. 

We are also definining a local function called pretty_print (which uses etree functionality) that just makes things easier for you to read! 

The execution is in the manager.connect function. It takes in the IP, the PORT, the USER, and the PASS you defined in the preceeding code snippet, and sets up the connetion to the router via NETCONF. As soon as you execute this, you are connected to this router via NETCONF. (congratulations)


### Capabilities Discovery

Let's look at the capabilities presented by the router you connected to:

In [11]:
for item in iter(m.server_capabilities):
    print(item)

urn:ietf:params:xml:ns:yang:smiv2:CISCO-IPSLA-ECHO-MIB?module=CISCO-IPSLA-ECHO-MIB&revision=2007-08-16
urn:cisco:params:xml:ns:yang:cisco-qos-common?module=cisco-qos-common&revision=2015-05-09
http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z?module=SNMPv2-MIB&revision=2002-10-16
urn:cisco:params:xml:ns:yang:cisco-environment?module=cisco-environment&revision=2015-04-09
http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z?module=SNMP-NOTIFICATION-MIB&revision=2002-10-14
urn:ietf:params:xml:ns:yang:smiv2:CISCO-CBP-TARGET-TC-MIB?module=CISCO-CBP-TARGET-TC-MIB&revision=2006-03-24
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:xml:ns:yang:smiv2:DISMAN-EVENT-MIB?module=DISMAN-EVENT-MIB&revision=2000-10-16
urn:ietf:params:xml:ns:yang:smiv2:CISCO-TC?module=CISCO-TC&revision=2011-11-11
http://tail-f.com/ns/webui?module=tailf-webui&revision=2013-03-07
urn:ietf:params:xml:ns:yang:smiv2:CISCO-PRODUCTS-MIB?module=CISCO-PRODUCTS-MIB&revision=2014-11-06
urn:ietf:params

Ok, that is alot of data. What is it anyway? 

It is the initial exchange the NETCONF server (aka the router) has with its client (this notebook). Any piece of software (like this notebook, or a controller, etc.) can use this information to understand what the device can do, and possibly more importantly, what it cannot do!

So now, let's tidy up the above and look, initially, at all the base netconf capabilities:

In [12]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=report-all
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0


As you might can tell, the code here just filters what was in the preceeding code snippet and makes it look more presentable to you. Either way, this looks better, but what does it mean? Well, if this were a real software program, this piece of software would already now know what the 3850 supports in terms of NETCONF. 

For example, the software would know that it supports a writable-running configuration store (via urn:ietf:params:netconf:capability:writable-running:1.0). This is defined by NETCONF, and represents the device's running configuration (you know, what you see when you do 'sho run' on CLI). But since you do NOT see a writable-startup message (which would look like urn:ietf:params:netconf:capability:startup:1.0), this lets you know that your software cannot not save changes to startup config using the standard <copy-config> operation and that you cannot use <edit-config> against the startup config. There are Cisco-specific primitives to work with startup config, but we will save that for later!

Further, since you (or your software) do NOT see a "candiate" capability (which would look like urn:ietf:params:netconf:capability:candidate:1.0), you know that your software cannot stage multiple changes to the configuration (as a separate scratchpad) to be executed later as a single transaction. IOS-XE 16.3 does not yet support this type of capability.

So far, it seems like NETCONF executes configuration just like CLI does (via writable-running) which modifies things directly into the running configuration, but with one **KEY** difference, which is support for the capability urn:ietf:params:netconf:capability:rollback-on-error:1.0. The support for this capability is perhaps one of the most compelling reasons to adopt NETCONF/YANG. This capability tells us that the router guarantees that any one <edit-config> operation will be carried out atomically. IOW, either it **all** happen or **none** of it happens. Just think about that for a moment in the context of how you have to interact with routers and switches over CLI or SNMP!

These are just a few small examples, but also crucial to understand how easily things can now be coded once these initial exchanges take place (and are understood) between client and server (as opposed to the human and the router).

For more details on what these capabilites mean, please reference RFC-6241.

In conclusion, the capabilities exchange here, lets you (or your piece of software) understand **HOW** you can manipulate the router.

### Models Discovery
So now, let's tidy up the above once more, and look at the capabilities that are related to model support:

In [13]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))
        
        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))
            
        deviations = re.search('deviations=([a-zA-Z0-9_\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))
            
        features = re.search('features=([a-zA-Z0-9_\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

CISCO-IPSLA-ECHO-MIB
  revision = 2007-08-16
cisco-qos-common
  revision = 2015-05-09
SNMPv2-MIB
  revision = 2002-10-16
cisco-environment
  revision = 2015-04-09
SNMP-NOTIFICATION-MIB
  revision = 2002-10-14
CISCO-CBP-TARGET-TC-MIB
  revision = 2006-03-24
DISMAN-EVENT-MIB
  revision = 2000-10-16
CISCO-TC
  revision = 2011-11-11
tailf-webui
  revision = 2013-03-07
CISCO-PRODUCTS-MIB
  revision = 2014-11-06
CISCO-UBE-MIB
  revision = 2010-11-29
CISCO-AAA-SESSION-MIB
  revision = 2006-03-21
tailf-common-monitoring
  revision = 2013-06-14
cisco-process-cpu
  revision = 2015-04-09
CISCO-RF-MIB
  revision = 2005-09-01
ENTITY-MIB
  revision = 2005-08-10
CISCO-IPMROUTE-MIB
  revision = 2005-03-07
cisco-efp-stats
  revision = 2015-07-07
cisco-qos-action-qlimit-cfg
  revision = 2015-05-09
  features = qos-grp-based-queuing-support,mpls-exp-based-queuing-support,disc-class-based-queuing-support
CISCO-BULK-FILE-MIB
  revision = 2002-06-10
iana-crypt-hash
  revision = 2014-04-04
  features = crypt

Don't let the code above scare you. We are once more, just using it to make the code snippet from two executions ago (which is the initial capabilities exchange) look more presentable to you. What it techncially does is that it pulls out the model names themselves, the deviations, the versions and the features. Just data parsing here ;-).

Parsing out the models, let's you know what aspects of configuration you have available to work with.

So let's filter it further to explain an example ...

In [14]:
import re

for c in m.server_capabilities:
    model = re.search('module=(ietf-interfaces)&', c)
    if model is not None:
        print("{}".format(model.group(1)))
        
        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))
            
        deviations = re.search('deviations=([a-zA-Z0-9_\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))
            
        features = re.search('features=([a-zA-Z0-9_\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))
            
        break

ietf-interfaces
  revision = 2014-05-08
  deviations = ietf-ip-devs
  features = pre-provisioning,if-mib,arbitrary-names


Once more, the code above just parses the exchange such that we only see information from one model. In this case, ietf-interfaces. This model has a revision, indicating when it was specified.

Importantly, it also has features and deviations.

**Features** identify which *optional* sections of the YANG model the platform advertising support for ietf-interfaces actually implements.

The presence of **deviations** let us know that Cisco has deviated the model from the standard in some way (as ietf-interfaces (you might guess) is a standard model defined by the IETF). How has Cisco deviated it? We need to go look at the ietf-ip-devs model of course, because YANG provides a standard way to describe these differences!

Ok, so here's some quick code that does that. We could have gone to github, or loaded pyang, but remember, we are connected to our router via NETCONF. 

In [15]:
SCHEMA_TO_GET = 'ietf-ip-devs'
c = m.get_schema(SCHEMA_TO_GET)
print c.data

module ietf-ip-devs {
  namespace "http://cisco.com/ns/ietf-ip/devs";

  prefix ip-devs;

  import ietf-interfaces {
     prefix if;
  }

  import ietf-ip {
     prefix ip;
  }

  organization
    "Cisco Systems, Inc.";

  contact
    "Cisco Systems, Inc.
    Customer Service

    Postal: 170 W Tasman Drive
    San Jose, CA 95134

    Tel: +1 1800 553-NETS

    E-mail: cs-yang@cisco.com";

  description
    "This module defines deviation statements for ietf-ip module.";

  revision 2016-08-10 {
    description
      "Updated deviation statements for 16.3.2 release.";
  }

  revision 2015-09-11 {
    description
      "Initial Revision";

    reference
      "RFC 6020: YANG - A Data Modeling Language for the
       Network Configuration Protocol (NETCONF)";
  }

  deviation /if:interfaces/if:interface/ip:ipv4/ip:enabled {
      deviate not-supported;
      description  "Not supported in IOS-XE 3.17 release.";
  }

  deviation /if:interfaces/if:interface/ip:ipv4/ip:forwarding {
      dev

So we just asked our router for this model directly with the code snippet above. Without going into many details here, the model above tells us how Cisco has deviated form the standard model, and IOS-XE technically does not support a handful of things compared to the standard-model. This is important for software to understand, and NETCONF/YANG supports the standardized exchange of this information.

Further, we are not supporting some of our own augmentations to the ietf-interfaces model. The augmentations of the ietf-interfaces model can be seen by looking at the ietf-ip model (which is shown to be imported by the deviations above). ieft-ip augments ietf-interfaces itself and is defined by RFC-7277

Now, remember this from earlier?

``` html
ietf-interfaces
  revision = 2014-05-08
  deviations = ietf-ip-devs
  features = pre-provisioning,if-mib,arbitrary-names```
  
It told our software the deviations from the standard model, but what about the features?

Pre-provisioning is a feature defined by RFC-7233. This allows software to configure interfaces that might not yet be operable on the system.

IF-MIB is a feature defined by RFC-2863. This allows software to know how the the interface should be managed.

Arbitrary-names is a feature defined by RFC-7233 as well. This allows software to more easily deal with virtual interfaces in addition to interfaces defined by the hardware resident on the platforms. VLANs are a typical example.

The YANG model designers identified these features as optional, and YANG itself provides the mechanism to tell the developer which optional features a platform implements.

### Conclusion
NETCONF presents a number of useful primitives that we looked at, but the two most important are Capabilities Discovery, which tells you HOW you can work with the box, and Model Discovery, which tells you WHAT features the box supports.

With the primitives we ran through, you can do basic model discovery to get the big picture of what you have to work with, understand what optional features are supported, and understand which parts of models are perhaps not supported.

Products such as NSO and open source projects like ODL or the YDK can use these capabilities to work with devices in a much more reliable way. So can the software your customers write.

And remember, we haven't even **DONE** anything yet. Stay tuned for more ...